In [6]:
import spacy
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [18]:
import pandas as pd

In [8]:
df=pd.read_csv("https://raw.githubusercontent.com/mdsohaib/BBC-News-Classification/master/bbc-text.csv").head(500)

In [9]:
df.shape

(500, 2)

In [10]:
df.head(5)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [11]:
df["category"].unique()

array(['tech', 'business', 'sport', 'entertainment', 'politics'],
      dtype=object)

In [12]:
df["category"].value_counts()

category
business         128
sport            118
politics          97
entertainment     81
tech              76
Name: count, dtype: int64

In [13]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [15]:
df["category"]=encoder.fit_transform(df["category"])

In [16]:
df.head()

,category,text
0,4,tv future in the hands of viewers with home th...
1,0,worldcom boss left books alone former worldc...
2,3,tigers wary of farrell gamble leicester say ...
3,3,yeading face newcastle in fa cup premiership s...
4,1,ocean s twelve raids box office ocean s twelve...


In [17]:
df["category"].unique()

array([4, 0, 3, 1, 2])

In [20]:
def preprocess(text):
  doc=nlp(text)
  tokens=[token.lemma_ for token in doc if not token.is_stop and token.pos_ not in ["SPACE", "X"]]
  return " ".join(tokens)

In [23]:
df["text"]=df["text"].apply(preprocess)

In [24]:
df.head()

,category,text
0,4,tv future hand viewer home theatre system plas...
1,0,worldcom boss leave book worldcom boss bernie ...
2,3,tiger wary farrell gamble leicester rush make ...
3,3,yeade face newcastle fa cup premiership newcas...
4,1,ocean s raid box office ocean s crime caper se...


In [26]:
df["vector"]=df["text"].apply(lambda text: nlp(text).vector)

In [27]:
df.head()

,category,text,vector
0,4,tv future hand viewer home theatre system plas...,"[-0.38491005, 0.6905412, -1.8487853, -0.179093..."
1,0,worldcom boss leave book worldcom boss bernie ...,"[-0.51069367, -0.75568306, -2.3575122, -0.1846..."
2,3,tiger wary farrell gamble leicester rush make ...,"[-0.1727503, 0.5202814, -1.4264264, 0.2079478,..."
3,3,yeade face newcastle fa cup premiership newcas...,"[-2.2051911, -0.6096624, 0.651152, 0.017446635..."
4,1,ocean s raid box office ocean s crime caper se...,"[-1.386107, -1.2569575, -2.395381, -0.5982053,..."


In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(df["vector"].values, df["category"].values, test_size=0.2, random_state=42)

In [36]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.pipeline import Pipeline
import seaborn as sns

In [44]:
X_train=np.stack(X_train)
y_train=np.stack(y_train)

In [45]:
model=Pipeline([
    ("scaler",MinMaxScaler()),
    ("model",MultinomialNB())
])

In [46]:
model.fit(X_train,y_train)

Pipeline(steps=[('scaler', MinMaxScaler()), ('model', MultinomialNB())])

In [50]:
preds=model.predict(np.stack(X_test))

In [51]:
print(accuracy_score(y_test,preds))

0.87
